In [1]:
!pip install transformers datasets accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 

In [2]:
from datasets import load_dataset

dataset = load_dataset("medical_questions_pairs")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3048 [00:00<?, ? examples/s]

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext")
model = AutoModelForSequenceClassification.from_pretrained("microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext", num_labels=2)  # Adjust `num_labels` as needed


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
!pip install transformers datasets requests


In [5]:
import requests

def search_pubmed(query):
    url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term={query}&retmode=json"
    response = requests.get(url)
    data = response.json()
    return data['esearchresult']['idlist']

# Example search query
article_ids = search_pubmed("machine learning in healthcare")
print(article_ids)


['38943990', '38943093', '38942044', '38941973', '38941140', '38940180', '38939618', '38939409', '38939246', '38938583', '38937795', '38937744', '38937158', '38935998', '38935419', '38934793', '38934289', '38934029', '38933933', '38933326']


In [6]:
def fetch_abstracts(article_ids):
    abstracts = []
    for article_id in article_ids:
        url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&id={article_id}&retmode=json"
        response = requests.get(url)
        data = response.json()
        try:
            abstract = data['result'][article_id]['title'] + " " + data['result'][article_id]['source']
            abstracts.append(abstract)
        except KeyError:
            continue
    return abstracts

# Fetch abstracts
abstracts = fetch_abstracts(article_ids)
print(abstracts)


['Advances in AI-assisted biochip technology for biomedicine. Biomed Pharmacother', 'Machine learning algorithms for predicting COVID-19 mortality in Ethiopia. BMC Public Health', 'The potential role for artificial intelligence in fracture risk prediction. Lancet Diabetes Endocrinol', 'Using Artificial Intelligence to predict outcomes of operatively managed neck of femur fractures. Br J Hosp Med (Lond)', 'Considerations for Quality Control Monitoring of Machine Learning Models in Clinical Practice. JMIR Med Inform', 'TA-RNN: an attention-based time-aware recurrent neural network architecture for electronic health records. Bioinformatics', 'The novel hierarchical clustering approach using self-organizing map with optimum dimension selection. Health Care Sci', 'Early Recognition of Clinical Trajectories Using Machine Learning in Hospitalized Heart\xa0Failure Patients. JACC Adv', 'Artificial Intelligence and Machine Learning in Predicting the Response to Immunotherapy in Non-small Cell Lu

In [7]:
from transformers import pipeline

# Initialize the summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def summarize_abstracts(abstracts):
    combined_abstracts = " ".join(abstracts)
    summary = summarizer(combined_abstracts, max_length=512, min_length=30, do_sample=False)
    return summary[0]['summary_text']

# Get the summary
summary = summarize_abstracts(abstracts)
print(summary)


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your max_length is set to 512, but your input_length is only 448. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=224)


The potential role for artificial intelligence in fracture risk prediction. Advances in AI-assisted biochip technology for biomedicine. Machine learning algorithms for predicting COVID-19 mortality in Ethiopia.


In [8]:


import requests
from transformers import pipeline

def search_pubmed(query):
    url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term={query}&retmode=json"
    response = requests.get(url)
    data = response.json()
    return data['esearchresult']['idlist']

def fetch_abstracts(article_ids):
    abstracts = []
    for article_id in article_ids:
        url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&id={article_id}&retmode=json"
        response = requests.get(url)
        data = response.json()
        try:
            abstract = data['result'][article_id]['title'] + " " + data['result'][article_id]['source']
            abstracts.append(abstract)
        except KeyError:
            continue
    return abstracts

def summarize_abstracts(abstracts):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    combined_abstracts = " ".join(abstracts)
    summary = summarizer(combined_abstracts, max_length=512, min_length=30, do_sample=False)
    return summary[0]['summary_text']

# Example usage
query = "machine learning in healthcare"
article_ids = search_pubmed(query)
abstracts = fetch_abstracts(article_ids)
summary = summarize_abstracts(abstracts)
print(summary)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your max_length is set to 512, but your input_length is only 448. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=224)


The potential role for artificial intelligence in fracture risk prediction. Advances in AI-assisted biochip technology for biomedicine. Machine learning algorithms for predicting COVID-19 mortality in Ethiopia.


In [9]:
!pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.7 MB/s eta 0:00:00


In [10]:
import ipywidgets as widgets
from IPython.display import display, HTML
import requests
from transformers import pipeline

def search_pubmed(query):
    url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term={query}&retmode=json"
    response = requests.get(url)
    data = response.json()
    return data['esearchresult']['idlist']

def fetch_abstracts(article_ids):
    abstracts = []
    for article_id in article_ids:
        url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&id={article_id}&retmode=json"
        response = requests.get(url)
        data = response.json()
        try:
            abstract = data['result'][article_id]['title'] + " " + data['result'][article_id]['source']
            abstracts.append(abstract)
        except KeyError:
            continue
    return abstracts

def summarize_abstracts(abstracts):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    combined_abstracts = " ".join(abstracts)
    summary = summarizer(combined_abstracts, max_length=512, min_length=30, do_sample=False)
    return summary[0]['summary_text']

def on_button_clicked(b):
    query = text.value
    with out:
        out.clear_output()
        print("Searching PubMed...")
        article_ids = search_pubmed(query)
        print(f"Found {len(article_ids)} articles.")
        abstracts = fetch_abstracts(article_ids)
        if abstracts:
            print("Summarizing abstracts...")
            summary = summarize_abstracts(abstracts)
            print("Summary:")
            print(summary)
        else:
            print("No abstracts found.")

# Create a text input widget
text = widgets.Text(
    value='',
    placeholder='Enter your search query',
    description='Query:',
    disabled=False
)

# Create a button widget
button = widgets.Button(
    description='Search',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Search PubMed',
    icon='search' # (FontAwesome names without the `fa-` prefix)
)

# Output widget
out = widgets.Output()

# Set the button click event
button.on_click(on_button_clicked)

# Display the widgets
display(text, button, out)


Text(value='', description='Query:', placeholder='Enter your search query')

Button(description='Search', icon='search', style=ButtonStyle(), tooltip='Search PubMed')

Output()

In [11]:
!pip install gTTS pydub

In [14]:
import ipywidgets as widgets
from IPython.display import display, Audio
import requests
from transformers import pipeline
from gtts import gTTS
from pydub import AudioSegment
import io

def search_pubmed(query):
    url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term={query}&retmode=json"
    response = requests.get(url)
    data = response.json()
    return data['esearchresult']['idlist']

def fetch_abstracts(article_ids):
    abstracts = []
    for article_id in article_ids:
        url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&id={article_id}&retmode=json"
        response = requests.get(url)
        data = response.json()
        try:
            abstract = data['result'][article_id]['title'] + " " + data['result'][article_id]['source']
            abstracts.append(abstract)
        except KeyError:
            continue
    return abstracts

def summarize_abstracts(abstracts):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    combined_abstracts = " ".join(abstracts)
    summary = summarizer(combined_abstracts, max_length=512, min_length=30, do_sample=False)
    return summary[0]['summary_text']

def text_to_speech(text):
    tts = gTTS(text)
    audio_fp = io.BytesIO()
    tts.write_to_fp(audio_fp)
    audio_fp.seek(0)
    return audio_fp

def on_button_clicked(b):
    query = text.value
    with out:
        out.clear_output()
        print("Searching PubMed...")
        article_ids = search_pubmed(query)
        print(f"Found {len(article_ids)} articles.")
        abstracts = fetch_abstracts(article_ids)
        if abstracts:
            print("Summarizing abstracts...")
            summary = summarize_abstracts(abstracts)
            print("Summary:")
            print(summary)

            print("Converting summary to audio...")
            audio_fp = text_to_speech(summary)
            audio_segment = AudioSegment.from_file(audio_fp, format="mp3")
            audio_segment.export("summary_audio.mp3", format="mp3")
            display(Audio("summary_audio.mp3", autoplay=True))

            print("Please enter a question based on the summary:")
            question_widget = widgets.Text(
                value='',
                placeholder='Enter your question here',
                description='Question:',
                disabled=False
            )
            ask_button = widgets.Button(description='Ask', disabled=False, button_style='', tooltip='Ask the question', icon='question')

            def on_ask_button_clicked(a):
                with out:
                    print("Generating answer...")
                    qa_pipeline = pipeline("question-answering")
                    question = question_widget.value
                    answer = qa_pipeline(question=question, context=summary)
                    print(f"Q: {question}")
                    print(f"A: {answer['answer']}")

            ask_button.on_click(on_ask_button_clicked)
            display(question_widget, ask_button)
        else:
            print("No abstracts found.")

# Create a text input widget
text = widgets.Text(
    value='',
    placeholder='Enter your search query',
    description='Query:',
    disabled=False
)

# Create a button widget
button = widgets.Button(
    description='Search',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Search PubMed',
    icon='search' # (FontAwesome names without the `fa-` prefix)
)

# Output widget
out = widgets.Output()

# Set the button click event
button.on_click(on_button_clicked)

# Display the widgets
display(text, button, out)


Text(value='', description='Query:', placeholder='Enter your search query')

Button(description='Search', icon='search', style=ButtonStyle(), tooltip='Search PubMed')

Output()

In [21]:
import ipywidgets as widgets
from IPython.display import display, Audio
import requests
from transformers import pipeline
from gtts import gTTS
from pydub import AudioSegment
import io

def search_pubmed(query):
    url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term={query}&retmode=json"
    response = requests.get(url)
    data = response.json()
    return data['esearchresult']['idlist']

def fetch_abstracts(article_ids):
    abstracts = []
    for article_id in article_ids:
        url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&id={article_id}&retmode=json"
        response = requests.get(url)
        data = response.json()
        try:
            abstract = data['result'][article_id]['title'] + " " + data['result'][article_id]['source']
            abstracts.append(abstract)
        except KeyError:
            continue
    return abstracts

def summarize_abstracts(abstracts):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    combined_abstracts = " ".join(abstracts)
    summary = summarizer(combined_abstracts, max_length=512, min_length=30, do_sample=False)
    return summary[0]['summary_text']

def text_to_speech(text):
    tts = gTTS(text)
    audio_fp = io.BytesIO()
    tts.write_to_fp(audio_fp)
    audio_fp.seek(0)
    return audio_fp

def on_button_clicked(b):
    query = text.value
    with out:
        out.clear_output()
        print("Searching PubMed...")
        article_ids = search_pubmed(query)
        print(f"Found {len(article_ids)} articles.")
        abstracts = fetch_abstracts(article_ids)
        if abstracts:
            print("Summarizing abstracts...")
            summary = summarize_abstracts(abstracts)
            print("Summary:")
            print(summary)

            print("Converting summary to audio...")
            audio_fp = text_to_speech(summary)
            audio_segment = AudioSegment.from_file(audio_fp, format="mp3")
            audio_segment.export("summary_audio.mp3", format="mp3")
            display(Audio("summary_audio.mp3", autoplay=True))

            print("Please enter a question based on the summary:")
            question_widget = widgets.Text(
                value='',
                placeholder='Enter your question here',
                description='Question:',
                disabled=False
            )
            ask_button = widgets.Button(description='Ask', disabled=False, button_style='', tooltip='Ask the question', icon='question')

            def on_ask_button_clicked(a):
                with out:
                    print("Generating answer...")
                    qa_pipeline = pipeline("question-answering")
                    question = question_widget.value
                    answer = qa_pipeline(question=question, context=summary)
                    print(f"Q: {question}")
                    print(f"A: {answer['answer']}")

            ask_button.on_click(on_ask_button_clicked)
            display(question_widget, ask_button)
        else:
            print("No abstracts found.")

# Create a text input widget
text = widgets.Text(
    value='',
    placeholder='Enter your search query',
    description='Query:',
    disabled=False
)

# Create a button widget
button = widgets.Button(
    description='Search',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Search PubMed',
    icon='search' # (FontAwesome names without the `fa-` prefix)
)

# Output widget
out = widgets.Output()

# Set the button click event
button.on_click(on_button_clicked)

# Display the widgets
display(text, button, out)


Text(value='', description='Query:', placeholder='Enter your search query')

Button(description='Search', icon='search', style=ButtonStyle(), tooltip='Search PubMed')

Output()

In [28]:
import ipywidgets as widgets
from IPython.display import display, Audio
import requests
from transformers import pipeline
from gtts import gTTS
from pydub import AudioSegment
import io

def search_pubmed(query):
    url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term={query}&retmode=json"
    response = requests.get(url)
    data = response.json()
    return data['esearchresult']['idlist']

def fetch_abstracts(article_ids):
    abstracts = []
    for article_id in article_ids:
        url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&id={article_id}&retmode=json"
        response = requests.get(url)
        data = response.json()
        try:
            abstract = data['result'][article_id]['title'] + " " + data['result'][article_id]['source']
            abstracts.append(abstract)
        except KeyError:
            continue
    return abstracts

def summarize_abstracts(abstracts):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    combined_abstracts = " ".join(abstracts)
    summary = summarizer(combined_abstracts, max_length=512, min_length=30, do_sample=False)
    return summary[0]['summary_text']

def text_to_speech(text):
    tts = gTTS(text)
    audio_fp = io.BytesIO()
    tts.write_to_fp(audio_fp)
    audio_fp.seek(0)
    return audio_fp

def on_button_clicked(b):
    query = text.value
    with out:
        out.clear_output()
        print("Searching PubMed...")
        article_ids = search_pubmed(query)
        print(f"Found {len(article_ids)} articles.")
        abstracts = fetch_abstracts(article_ids)
        if abstracts:
            print("Summarizing abstracts...")
            summary = summarize_abstracts(abstracts)
            print("Summary:")
            print(summary)

            print("Converting summary to audio...")
            audio_fp = text_to_speech(summary)
            audio_segment = AudioSegment.from_file(audio_fp, format="mp3")
            audio_segment.export("summary_audio.mp3", format="mp3")
            display(Audio("summary_audio.mp3", autoplay=True))

            print("Please enter a question based on the summary:")
            ask_button = widgets.Button(description='Ask', disabled=False, button_style='',
                                        tooltip='Ask the question', icon='question')
            question_widget = widgets.Text(
                value='',
                placeholder='Enter your question here',
                description='Question:',
                disabled=False
            )

            display(question_widget, ask_button)
            def on_ask_button_clicked(a):
                with out:
                    print("Generating answer...")
                    qa_pipeline = pipeline("question-answering")
                    question = question_widget.value
                    answer = qa_pipeline(question=question, context=summary)
                    print(f"Q: {question}")
                    print(f"A: {answer['answer']}")

            ask_button.on_click(on_ask_button_clicked)

        else:
            print("No abstracts found.")

# Create a text input widget
text = widgets.Text(
    value='',
    placeholder='Enter your search query',
    description='Query:',
    disabled=False
)

# Create a button widget
button = widgets.Button(
    description='Search',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Search PubMed',
    icon='search' # (FontAwesome names without the `fa-` prefix)
)

# Output widget
out = widgets.Output()

# Set the button click event
button.on_click(on_button_clicked)

# Display the widgets
display(text, button, out)


Text(value='', description='Query:', placeholder='Enter your search query')

Button(description='Search', icon='search', style=ButtonStyle(), tooltip='Search PubMed')

Output()

In [29]:
!pip install transformers datasets requests gtts
!pip install pydub ipywidgets


In [30]:
import ipywidgets as widgets
from IPython.display import display, Audio, clear_output
import requests
from transformers import pipeline
from gtts import gTTS
from pydub import AudioSegment
import io
import torch

def search_pubmed(query):
    url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term={query}&retmode=json"
    response = requests.get(url)
    data = response.json()
    return data['esearchresult']['idlist']

def fetch_abstracts(article_ids):
    abstracts = []
    for article_id in article_ids:
        url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&id={article_id}&retmode=json"
        response = requests.get(url)
        data = response.json()
        try:
            abstract = data['result'][article_id]['title'] + " " + data['result'][article_id]['source']
            abstracts.append(abstract)
        except KeyError:
            continue
    return abstracts

def summarize_abstracts(abstracts):
    device = 0 if torch.cuda.is_available() else -1
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=device)
    combined_abstracts = " ".join(abstracts)
    summary = summarizer(combined_abstracts, max_length=512, min_length=30, do_sample=False)
    return summary[0]['summary_text']

def text_to_speech(text):
    tts = gTTS(text)
    audio_fp = io.BytesIO()
    tts.write_to_fp(audio_fp)
    audio_fp.seek(0)
    return audio_fp

def on_button_clicked(b):
    query = text.value
    with out:
        clear_output()
        print("Searching PubMed...")
        article_ids = search_pubmed(query)
        print(f"Found {len(article_ids)} articles.")
        abstracts = fetch_abstracts(article_ids)
        if abstracts:
            print("Summarizing abstracts...")
            summary = summarize_abstracts(abstracts)
            print("Summary:")
            print(summary)

            print("Converting summary to audio...")
            audio_fp = text_to_speech(summary)
            audio_segment = AudioSegment.from_file(audio_fp, format="mp3")
            audio_segment.export("summary_audio.mp3", format="mp3")
            display(Audio("summary_audio.mp3", autoplay=True))

            print("Please enter a question based on the summary:")
            question_widget = widgets.Text(
                value='',
                placeholder='Enter your question here',
                description='Question:',
                disabled=False
            )
            ask_button = widgets.Button(description='Ask', disabled=False, button_style='', tooltip='Ask the question', icon='question')

            def on_ask_button_clicked(a):
                with out:
                    clear_output(wait=True)
                    print("Generating answer...")
                    device = 0 if torch.cuda.is_available() else -1
                    qa_pipeline = pipeline("question-answering", device=device)
                    question = question_widget.value
                    answer = qa_pipeline(question=question, context=summary)
                    print(f"Q: {question}")
                    print(f"A: {answer['answer']}")
                    display(question_widget, ask_button)

            ask_button.on_click(on_ask_button_clicked)
            display(question_widget, ask_button)
        else:
            print("No abstracts found.")

# Create a text input widget
text = widgets.Text(
    value='',
    placeholder='Enter your search query',
    description='Query:',
    disabled=False
)

# Create a button widget
button = widgets.Button(
    description='Search',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Search PubMed',
    icon='search' # (FontAwesome names without the `fa-` prefix)
)

# Output widget
out = widgets.Output()

# Set the button click event
button.on_click(on_button_clicked)

# Display the widgets
display(text, button, out)


Text(value='', description='Query:', placeholder='Enter your search query')

Button(description='Search', icon='search', style=ButtonStyle(), tooltip='Search PubMed')

Output()